# 一般化線形モデル　ポアソン回帰モデル
- Generalized Linear Models https://www.statsmodels.org/stable/glm.html
- statsmodels.formula.api.glm https://www.statsmodels.org/dev/generated/statsmodels.formula.api.glm.html

$\lambda = \exp(\beta_0+ \beta_1)$

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import scipy as sp
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import utils

import matplotlib.pyplot as plt
#import japanize_matplotlib
%matplotlib inline

## ポアソン回帰モデルの意義を見る


In [ ]:
np.random.seed(123)
xend, N  = 10.0, 100
b0, b1 = 0.1, 0.4
x = np.linspace(0.0, xend, N)
y0 = b0 + b1*x
y = y0 + sp.stats.norm.rvs(loc=0, scale=0.2, size=N)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,3))
ax[0].scatter(x,y, c='w', ec='k', label='y')
ax[0].plot(x,y0, c='k', label='b0+b1*x')
ax[0].set_xlabel('x', fontsize=12)
ax[0].set_ylabel('y', fontsize=12)
ax[0].grid()
ax[0].legend(fontsize=12)

lamb = np.exp(b0 + b1*x)
ypoisson = sp.stats.poisson.rvs(mu=lamb, size=N)

ax[1].scatter(x,ypoisson, c='w', ec='k', label='y')
ax[1].plot(x,lamb, c='k', label='lambda=exp(b0+b1*x)')
ax[1].set_xlabel('x', fontsize=12)
ax[1].set_ylabel('y', fontsize=12)
ax[1].grid()
ax[1].legend(fontsize=12)


#plt.savefig('fig_REG_GLM_exam.pdf', bbox_inches='tight')
plt.show()

#### 上の左図
$ y_i = \beta_0 + \beta_1 x_i + \varepsilon_i,  \hspace{3mm} \varepsilon \sim N(0, \sigma^2)$，
ここで，$ E [ y_i ] = \beta_0 + \beta_1 x_i $より，$y_i$の平均は$x_i$に依存して変化する。
#### 上の右図
$\lambda_i = \exp(\beta_0 + \beta_1 x_i)$より，平均$\lambda_i$は$x_i$に依存して指数状に変化。


In [ ]:
xend, N  = 10.0, 20
b0, b1 = 0.5, 0.2

In [ ]:
np.random.seed(123)
x = np.linspace(0.0, xend, N)
lamb = np.exp(b0 + b1*x)
y1 = sp.stats.poisson.rvs(mu=lamb, size=N)
y2 = sp.stats.poisson.rvs(mu=lamb, size=N)
y3 = sp.stats.poisson.rvs(mu=lamb, size=N)

In [ ]:
fig = plt.subplots(figsize=(5,3))
plt.scatter(x,y1, marker='o', c='w', ec='k', label='y1')
plt.scatter(x,y2, marker='^', c='w', ec='k', label='y2')
plt.scatter(x,y3, marker='s', c='w', ec='k', label='y3')
#plt.plot(x,lamb, c='k', label='b0+b1*x')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
#plt.xlim(-0.5,20)
plt.grid()
plt.legend(fontsize=12)
#plt.savefig('fig_REG_GLM_Poisson01.pdf', bbox_inches='tight')
plt.show()

## 三つのデータ全てを使用

In [ ]:
xall = np.concatenate([x,x,x])
yall = np.concatenate([y1, y2, y3])

In [ ]:
plt.plot(yall, marker='o', c='k')

In [ ]:
df = pd.DataFrame({'x':xall, 'y':yall})
glm_model = 'y ~ x'
result1 = smf.glm(formula=glm_model, data=df, family=sm.families.Poisson(link=sm.families.links.log())).fit()
print(result1.summary())

#### 相対誤差

In [ ]:
print('Relative error, b0:',np.abs( (result1.params[0]-b0)/b0),'  b1:',np.abs( (result1.params[1]-b1)/b1))

In [ ]:
lambda_pred = np.exp(result1.params[0] + result1.params[1]*x)

fig = plt.subplots(figsize=(5,3))
plt.plot(x,lambda_pred, c='k')
plt.scatter(x,y1, marker='o', c='w', ec='k', label='y1')
plt.scatter(x,y2, marker='^', c='w', ec='k', label='y2')
plt.scatter(x,y3, marker='s', c='w', ec='k', label='y3')
#plt.plot(x,lamb, c='k', label='b0+b1*x')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
#plt.xlim(-0.5,20)
plt.grid()
plt.legend(fontsize=12)
#plt.savefig('fig_REG_GLM_Poisson03.pdf', bbox_inches='tight')
plt.show()

## ｘとyの対応関係を保ったままシャッフルする

In [ ]:
id = np.random.permutation(len(xall))
xall2 = xall[id]
yall2 = yall[id]

fig = plt.subplots(figsize=(5,3))
plt.plot(yall, marker='x', c='lightgray', linestyle='-', label='Before')
plt.plot(yall2, marker='o', c='w', mec='k', linestyle='None', label='Shuffled')
plt.legend()

#plt.savefig('fig_REG_GLM_PoissonShuffle.pdf', bbox_inches='tight')
plt.show()

In [ ]:
df = pd.DataFrame({'x':xall2, 'y':yall2})
glm_model = 'y ~ x'
result2 = smf.glm(formula=glm_model, data=df, family=sm.families.Poisson(link=sm.families.links.log())).fit()
print(result2.summary())

#### 相対誤差
先の結果と最小桁数のあたりと異なっているのは，データの計算する順番が異なるため，数値計算における誤差と考えられ，これを無視すれば，同じパラメータ推定結果が得られていると言える。

In [ ]:
print('Relative error, b0:',np.abs( (result2.params[0]-b0)/b0),'  b1:',np.abs( (result2.params[1]-b1)/b1))

## y1だけ

In [ ]:
df = pd.DataFrame({'x':x, 'y':y1})
glm_model = 'y ~ x'
result_one = smf.glm(formula=glm_model, data=df, family=sm.families.Poisson(link=sm.families.links.log())).fit()
print(result_one.summary())

#### 相対誤差

In [ ]:
print('Relative error, b0:',np.abs( (result_one.params[0]-b0)/b0),'  b1:',np.abs( (result_one.params[1]-b1)/b1))

#### データ数Nを増やす

In [ ]:
N = 100

np.random.seed(123)
x = np.linspace(0.0, xend, N)
lamb = np.exp(b0 + b1*x)
y1 = sp.stats.poisson.rvs(mu=lamb, size=N)
y2 = sp.stats.poisson.rvs(mu=lamb, size=N)
y3 = sp.stats.poisson.rvs(mu=lamb, size=N)

fig = plt.subplots(figsize=(5,3))
plt.scatter(x,y1, marker='o', c='w', ec='k', label='y1')
plt.scatter(x,y2, marker='^', c='w', ec='k', label='y2')
plt.scatter(x,y3, marker='s', c='w', ec='k', label='y3')
#plt.plot(x,lamb, c='k', label='b0+b1*x')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
#plt.xlim(-0.5,20)
plt.grid()
plt.legend(fontsize=12)
#plt.savefig('fig_REG_GLM_PoissonMoreN.pdf', bbox_inches='tight')
plt.show()

In [ ]:
xall_more = np.concatenate([x,x,x])
yall_more = np.concatenate([y1, y2, y3])
df = pd.DataFrame({'x':xall_more, 'y':yall_more})
glm_model = 'y ~ x'
result_more = smf.glm(formula=glm_model, data=df, family=sm.families.Poisson(link=sm.families.links.log())).fit()
print(result_more.summary())

#### 相対誤差

In [ ]:
print('Relative error, b0:',np.abs( (result_more.params[0]-b0)/b0),'  b1:',np.abs( (result_more.params[1]-b1)/b1))

#### 結果のまとめ

３つのデータ：　Relative error, b0: 0.04651349715997477   b1: 0.04160179905478803<br>
シャフル　　：　Relative error, b0: 0.04651349715997455   b1: 0.041601799054788446<br>
ｙ１だけ　　；　Relative error, b0: 0.38501820824027844   b1: 0.07156391574263607<br>
データ数増す：　Relative error, b0: 0.015208882768656062   b1: 0.032686569622117356

## $z= \beta_0$の例

In [ ]:
np.random.seed(123)
xend, N  = 10.0, 100
b0 = 5
x = np.linspace(0.0, xend, N)
lamb = np.exp(b0)
ypoisson = sp.stats.poisson.rvs(mu=lamb, size=N)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,3))
plt.scatter(x,ypoisson, c='w', ec='k', label='y')
plt.grid()


In [ ]:
df = pd.DataFrame({'x':x, 'y':ypoisson})
glm_model = 'y ~ x'
result_b0 = smf.glm(formula=glm_model, data=df, family=sm.families.Poisson(link=sm.families.links.log())).fit()
print(result_b0.summary())

ポアソン分布の平均は，$E[y]=\lambda$であり，これと標本平均とを比較する。

In [ ]:
print('lambda = exp(b0)=', np.exp(b0))
print('Mean of y =      ',df.y.mean())